In [28]:
import requests
import pandas as pd
import json

EQUITY_KEYWORDS = ['diabetes', 'obesity', 'stroke', 'heart', 'blood pressure']


url = "https://chronicdata.cdc.gov/resource/cwsq-ngmh.json"
params = {"stateabbr": "MA", "$limit": "2000"}
response = requests.get(url, params=params)
ma_health_data = response.json()

print(f"Retrieved {len(ma_health_data)} health records from CDC PLACES")



Retrieved 2000 health records from CDC PLACES


In [29]:
def get_census_data():
    census_url = "https://api.census.gov/data/2022/acs/acs5"
    params = {
        "get": "B19013_001E,B25003_003E,B08301_010E,B15003_022E,B25001_001E,B17010_002E",
        "for": "county:*",
        "in": "state:25"
    }
    
    response = requests.get(census_url, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        
         
        df.columns = ['median_income', 'renter_units', 'transit_users', 
                     'college_grads', 'total_units', 'poverty_households',
                     'state_code', 'county_code']
        
         
        numeric_cols = df.columns[:-2]
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
         
        county_names = {
            '001': 'Barnstable', '003': 'Berkshire', '005': 'Bristol', '007': 'Dukes',
            '009': 'Essex', '011': 'Franklin', '013': 'Hampden', '015': 'Hampshire', 
            '017': 'Middlesex', '019': 'Nantucket', '021': 'Norfolk', '023': 'Plymouth',
            '025': 'Suffolk', '027': 'Worcester'
        }
        df['county_name'] = df['county_code'].map(county_names)
        df['poverty_rate'] = (df['poverty_households'] / df['total_units'] * 100).round(2)
        
        return df
    return None

social_data = get_census_data()
print(f"Retrieved social data for {len(social_data)} counties from Census ACS")

Retrieved social data for 14 counties from Census ACS


In [30]:
health_records = []
for record in ma_health_data:
    try:
        health_records.append({
            'year': record.get('year', ''),
            'county': record.get('countyname', ''),
            'measure': record.get('measure', ''),
            'value': float(record.get('data_value', 0)) if record.get('data_value') else None,
            'confidence_low': float(record.get('low_confidence_limit', 0)) if record.get('low_confidence_limit') else None,
            'confidence_high': float(record.get('high_confidence_limit', 0)) if record.get('high_confidence_limit') else None,
            'population': int(record.get('totalpopulation', 0)) if record.get('totalpopulation') else None
        })
    except:
        pass

health_df = pd.DataFrame(health_records)
print(f"Processed {len(health_df)} health records")

Processed 2000 health records


In [31]:
def analyze_health_equity(health_df, social_df):
   
    
  
    merged = pd.merge(
        health_df.assign(county_clean=health_df['county'].str.title().str.strip()),
        social_df.assign(county_clean=social_df['county_name'].str.title().str.strip()),
        on='county_clean', how='inner'
    )
    
   
    conditions = EQUITY_KEYWORDS
    results = {}
    
    for condition in conditions:
        condition_data = merged[merged['measure'].str.contains(condition, case=False, na=False)]
        if len(condition_data) > 0:
            poverty_corr = condition_data['poverty_rate'].corr(condition_data['value'])
            income_corr = condition_data['median_income'].corr(condition_data['value'])
            
             
            county_avg = condition_data.groupby('county_clean').agg({
                'value': 'mean',
                'poverty_rate': 'first',
                'median_income': 'first'
            }).round(2)
            
            results[condition] = {
                'records': len(condition_data),
                'poverty_correlation': poverty_corr,
                'income_correlation': income_corr,
                'county_data': county_avg
            }
    
    return results


analysis_results = analyze_health_equity(health_df, social_data)

In [32]:
print("\nHealth Equity Analysis Results")
print("=" * 50)

for condition, data in analysis_results.items():
    print(f"\n{condition}:")
    print(f"  Records: {data['records']}")
    print(f"  Poverty correlation: {data['poverty_correlation']:.3f}")
    print(f"  Income correlation: {data['income_correlation']:.3f}")


disparity_data = []
for condition, data in analysis_results.items():
    if len(data['county_data']) > 1:
        rates = data['county_data']['value']
        disparity_data.append({
            'condition': condition,
            'disparity_ratio': round(rates.max() / rates.min(), 2),
            'rate_range': round(rates.max() - rates.min(), 1)
        })

disparity_df = pd.DataFrame(disparity_data).sort_values('disparity_ratio', ascending=False)

print(f"\nHealth Disparities by Condition:")
print(disparity_df.to_string(index=False))


high_risk_counties = social_data[social_data['poverty_rate'] > social_data['poverty_rate'].median()]
print(f"\nHigh Social Vulnerability Counties:")
print(high_risk_counties[['county_name', 'poverty_rate', 'median_income']].sort_values('poverty_rate', ascending=False))


Health Equity Analysis Results

diabetes:
  Records: 45
  Poverty correlation: -0.049
  Income correlation: -0.293

obesity:
  Records: 73
  Poverty correlation: 0.517
  Income correlation: -0.395

stroke:
  Records: 130
  Poverty correlation: 0.108
  Income correlation: -0.369

heart:
  Records: 45
  Poverty correlation: -0.441
  Income correlation: 0.252

blood pressure:
  Records: 88
  Poverty correlation: -0.122
  Income correlation: 0.055

Health Disparities by Condition:
     condition  disparity_ratio  rate_range
        stroke             1.75         1.6
       obesity             1.69        15.5
         heart             1.54         3.6
      diabetes             1.18         1.7
blood pressure             1.16         8.5

High Social Vulnerability Counties:
   county_name  poverty_rate  median_income
6      Hampden          6.49          66619
12     Suffolk          5.30          87669
2      Bristol          5.25          80628
4        Essex          4.37          94

In [33]:
dashboard_data = []
for _, county_row in social_data.iterrows():
    for condition, data in analysis_results.items():
        county_data = data['county_data']
        if county_row['county_name'].title() in county_data.index:
            dashboard_data.append({
                'county': county_row['county_name'],
                'condition': condition,
                'prevalence_rate': county_data.loc[county_row['county_name'].title(), 'value'],
                'poverty_rate': county_row['poverty_rate'],
                'median_income': county_row['median_income']
            })

dashboard_df = pd.DataFrame(dashboard_data)
dashboard_df.to_csv('ma_health_equity_analysis.csv', index=False)

print(f"\nExported: ma_health_equity_analysis.csv ({len(dashboard_df)} records)")


Exported: ma_health_equity_analysis.csv (37 records)


In [34]:

years = {}
for record in ma_health_data:
    year = record.get('year', 'Unknown')
    years[year] = years.get(year, 0) + 1

print("Years in health dataset:")
for year, count in sorted(years.items()):
    print(f"  {year}: {count} records ({count/len(ma_health_data)*100:.1f}%)")

print(f"\nTotal records: {len(ma_health_data)}")


if 'health_df' in locals():
    print(f"\nYears in processed DataFrame:")
    year_counts = health_df['year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"  {year}: {count} records ({count/len(health_df)*100:.1f}%)")

Years in health dataset:
  2021: 177 records (8.8%)
  2022: 1823 records (91.1%)

Total records: 2000

Years in processed DataFrame:
  2021: 177 records (8.8%)
  2022: 1823 records (91.1%)


In [35]:
print(f"\nSummary:")
print(f"• Counties analyzed: {len(social_data)}")
print(f"• Health conditions: {len(analysis_results)}")
print(f"• Strong correlations (|r| > 0.3): {sum(1 for data in analysis_results.values() if abs(data['poverty_correlation']) > 0.3 or abs(data['income_correlation']) > 0.3)}")
print(f"• Largest disparity: {disparity_df.iloc[0]['condition']} ({disparity_df.iloc[0]['disparity_ratio']}x)")

print(f"\nAnalysis complete.")


Summary:
• Counties analyzed: 14
• Health conditions: 5
• Strong correlations (|r| > 0.3): 3
• Largest disparity: stroke (1.75x)

Analysis complete.


In [36]:
 
import folium
from folium import plugins
import numpy as np


def create_county_summary(health_df, social_df, analysis_results):
    """Prepare county-level data for mapping"""
    
    county_map_data = []
    
    for _, county_row in social_df.iterrows():
        county_name = county_row['county_name']
        
        
        county_data = {
            'county': county_name,
            'poverty_rate': county_row['poverty_rate'],
            'median_income': county_row['median_income'],
            'social_vulnerability_score': county_row['poverty_rate'] * 2 + (100000 - county_row['median_income']) / 1000
        }
        
    
        county_health = health_df[health_df['county'].str.contains(county_name, case=False, na=False)]
        if len(county_health) > 0:
            county_data['total_health_records'] = len(county_health)
            
            
            for condition in ['Diabetes', 'Obesity', 'Stroke', 'Heart']:
                condition_data = county_health[county_health['measure'].str.contains(condition, case=False, na=False)]
                if len(condition_data) > 0:
                    county_data[f'{condition.lower()}_rate'] = condition_data['value'].mean()
        
        county_map_data.append(county_data)
    
    return county_map_data

county_summary = create_county_summary(health_df, social_data, analysis_results)
map_df = pd.DataFrame(county_summary)

print(f"Prepared mapping data for {len(map_df)} counties")
print(f"Available metrics: {[col for col in map_df.columns if col not in ['county']]}")
 
# Massachusetts center coordinates
ma_center = [42.3601, -71.0589]

 
 

 
m1_fixed = folium.Map(
    location=[42.3601, -71.0589],
    zoom_start=8,
    tiles='CartoDB positron'
)

 
county_coords = {
    'Hampden': [42.1015, -72.5898],
    'Suffolk': [42.3601, -71.0589], 
    'Bristol': [41.8321, -71.1243],
    'Essex': [42.6619, -70.9342],
    'Worcester': [42.3126, -71.7611],
    'Franklin': [42.6042, -72.6851],
    'Berkshire': [42.3118, -73.1822],
    'Plymouth': [42.0624, -70.6631],
    'Middlesex': [42.4565, -71.2356],
    'Norfolk': [42.1751, -71.1956],
    'Barnstable': [41.6819, -70.1636],
    'Dukes': [41.3815, -70.6472],
    'Nantucket': [41.2835, -70.0995]
}

for _, row in map_df.iterrows():
    
    if row['poverty_rate'] > 5.0:
        color = 'red'
        risk_level = 'High Risk'
    elif row['poverty_rate'] > 3.0:
        color = 'orange' 
        risk_level = 'Medium Risk'
    else:
        color = 'green'
        risk_level = 'Low Risk'
    
    
    popup_text = f"""
    <b>{row['county']} County</b><br>
    Risk Level: <b>{risk_level}</b><br>
    Poverty Rate: {row['poverty_rate']:.1f}%<br>
    Median Income: ${row['median_income']:,}<br>
    Health Records: {row.get('total_health_records', 'N/A')}
    """
    
    if row['county'] in county_coords:
        folium.CircleMarker(
            location=county_coords[row['county']],
            radius=row['poverty_rate'] * 3,
            popup=popup_text,
            color=color,
            fillColor=color,
            fillOpacity=0.7,
            weight=2
        ).add_to(m1_fixed)

 
title_html = '''
<h3 align="center" style="font-size:20px"><b>Massachusetts Counties: Social Vulnerability</b></h3>
<p align="center">Circle size = Poverty rate | Color = Risk level</p>
'''
m1_fixed.get_root().html.add_child(folium.Element(title_html))

 
# Just the fixed legend HTML with increased height
legend_html = '''
<div style="position: fixed; 
            top: 100px; right: 20px; width: 220px; height: 150px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 15px; border-radius: 5px;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
<p style="margin: 0 0 10px 0; font-weight: bold;">Risk Levels</p>
<p style="margin: 5px 0;"><span style="color:red; font-size:16px;">●</span> High Risk (>5% poverty)</p>
<p style="margin: 5px 0;"><span style="color:orange; font-size:16px;">●</span> Medium Risk (3-5% poverty)</p>
<p style="margin: 5px 0;"><span style="color:green; font-size:16px;">●</span> Low Risk (<3% poverty)</p>
</div>
'''


m1_fixed.get_root().html.add_child(folium.Element(legend_html))


print("Social Vulnerability Map")
 
print()
m1_fixed

 

m2 = folium.Map(
    location=ma_center,
    zoom_start=8,
    tiles='CartoDB positron'
)

# Focus on obesity since it had strongest correlation
for _, row in map_df.iterrows():
    if pd.notna(row.get('obesity_rate')):
        obesity_rate = row['obesity_rate']
        
        
        if obesity_rate > 30:
            color = 'darkred'
            risk_level = 'Very High'
        elif obesity_rate > 25:
            color = 'red'
            risk_level = 'High'  
        elif obesity_rate > 20:
            color = 'orange'
            risk_level = 'Medium'
        else:
            color = 'green'
            risk_level = 'Low'
        
        popup_text = f"""
        <b>{row['county']} County</b><br>
        Obesity Rate: <b>{obesity_rate:.1f}%</b><br>
        Risk Level: {risk_level}<br>
        Poverty Rate: {row['poverty_rate']:.1f}%<br>
        Median Income: ${row['median_income']:,}
        """
        
        if row['county'] in county_coords:
            folium.CircleMarker(
                location=county_coords[row['county']],
                radius=obesity_rate / 2,   
                popup=popup_text,
                color=color,
                fillColor=color,
                fillOpacity=0.8,
                weight=2
            ).add_to(m2)

 
title_html2 = '''
<h3 align="center" style="font-size:20px"><b>Massachusetts Counties: Obesity Rates</b></h3>
<p align="center">Circle size = Obesity rate | Color = Health risk level</p>
'''

obesity_legend_html = '''
<div style="position: fixed; 
            top: 100px; right: 20px; width: 240px; height: 150px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 15px; border-radius: 5px;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
<p style="margin: 0 0 10px 0; font-weight: bold;">Obesity Risk Levels</p>
<p style="margin: 5px 0;"><span style="color:darkred; font-size:16px;">●</span> Very High (>30% obesity)</p>
<p style="margin: 5px 0;"><span style="color:red; font-size:16px;">●</span> High (25-30% obesity)</p>
<p style="margin: 5px 0;"><span style="color:orange; font-size:16px;">●</span> Medium (20-25% obesity)</p>
<p style="margin: 5px 0;"><span style="color:green; font-size:16px;">●</span> Low (<20% obesity)</p>
</div>
'''


m2.get_root().html.add_child(folium.Element(obesity_legend_html))
m2.get_root().html.add_child(folium.Element(title_html2))

print("Created Health Outcomes Map")

 


Prepared mapping data for 14 counties
Available metrics: ['poverty_rate', 'median_income', 'social_vulnerability_score', 'total_health_records', 'diabetes_rate', 'obesity_rate', 'stroke_rate', 'heart_rate']
Social Vulnerability Map

Created Health Outcomes Map


In [37]:
 
print("=== SOCIAL VULNERABILITY MAP ===")
print("Red = High risk counties (Hampden, Suffolk, Bristol)")
print("Circle size = poverty rate")
print()
m1_fixed

=== SOCIAL VULNERABILITY MAP ===
Red = High risk counties (Hampden, Suffolk, Bristol)
Circle size = poverty rate



In [38]:
print("=== OBESITY RATES MAP ===")
print("Shows geographic distribution of obesity (your strongest correlation)")
print("Circle size = obesity prevalence rate")
print()
m2

=== OBESITY RATES MAP ===
Shows geographic distribution of obesity (your strongest correlation)
Circle size = obesity prevalence rate



In [39]:


m3 = folium.Map(
    location=[42.2, -71.8],  # Centered more on MA interior
    zoom_start=8,
    tiles='CartoDB positron'
)

county_coords_ma_only = {
    'Hampden': [42.1015, -72.5898],      
    'Suffolk': [42.3601, -71.0589],       
    'Bristol': [41.9321, -71.0243],       
    'Essex': [42.6619, -70.9342],         
    'Worcester': [42.3126, -71.7611],     
    'Franklin': [42.6042, -72.6851],      
    'Berkshire': [42.4118, -73.1822],     
    'Plymouth': [42.1624, -70.6631],      
    'Middlesex': [42.4565, -71.2356],    
    'Norfolk': [42.2751, -71.1956],       
    'Barnstable': [41.7819, -70.1636],   
    'Dukes': [41.4815, -70.6472],         
    'Nantucket': [41.3835, -70.0995]      
}


for _, row in map_df.iterrows():
    if pd.notna(row.get('obesity_rate')) and row['county'] in county_coords_ma_only:
        obesity_rate = row['obesity_rate']
        poverty_rate = row['poverty_rate']
        
      
        obesity_score = (obesity_rate - 20) / 20 * 10  
        poverty_score = poverty_rate  
        combined_score = obesity_score + poverty_score
        
     
        if combined_score > 12:
            color = 'purple'
            risk_level = 'Dual High Risk'
        elif combined_score > 8:
            color = 'red' 
            risk_level = 'High Risk'
        elif combined_score > 5:
            color = 'orange'
            risk_level = 'Medium Risk'
        else:
            color = 'green'
            risk_level = 'Low Risk'
        
        popup_text = f"""
        <b>{row['county']} County, MA</b><br>
        <b style="color:{color};">Risk: {risk_level}</b><br>
        <hr style="margin: 5px 0;">
        • Obesity: <b>{obesity_rate:.1f}%</b><br>
        • Poverty: <b>{poverty_rate:.1f}%</b><br>
        • Score: {combined_score:.1f}<br>
        <hr style="margin: 5px 0;">
        <small>r=0.517 correlation evidence</small>
        """
        
        folium.CircleMarker(
            location=county_coords_ma_only[row['county']],
            radius=8 + combined_score * 0.4,  
            popup=folium.Popup(popup_text, max_width=250),
            color=color,
            fillColor=color,
            fillOpacity=0.8,
            weight=3
        ).add_to(m3)


title_html3 = '''
<h3 align="center" style="font-size:18px"><b>Massachusetts: Obesity-Poverty Correlation (r=0.517)</b></h3>
<p align="center">Purple = Strong correlation evidence | All dots within MA</p>
'''
m3.get_root().html.add_child(folium.Element(title_html3))


correlation_legend_complete = '''
<div style="position: fixed; 
            top: 70px; right: 10px; width: 290px; height: 295px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:13px; padding: 18px; border-radius: 5px;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
<p style="margin: 0 0 10px 0; font-weight: bold; color: #333; font-size: 14px;">Correlation Evidence</p>
<p style="margin: 4px 0 12px 0; font-size: 12px; color: #666;">r = 0.517 (Strong Relationship)</p>
<hr style="margin: 10px 0;">
<p style="margin: 8px 0 4px 0;"><span style="color:purple; font-size:20px;">●</span> <b>Dual High Risk</b></p>
<p style="margin: 2px 0 10px 24px; font-size: 11px; color: #666;">High obesity + High poverty</p>

<p style="margin: 8px 0 4px 0;"><span style="color:red; font-size:20px;">●</span> <b>High Risk</b></p>
<p style="margin: 2px 0 10px 24px; font-size: 11px; color: #666;">Either obesity or poverty elevated</p>

<p style="margin: 8px 0 4px 0;"><span style="color:orange; font-size:20px;">●</span> <b>Medium Risk</b></p>
<p style="margin: 2px 0 10px 24px; font-size: 11px; color: #666;">Moderate levels</p>

<p style="margin: 8px 0 4px 0;"><span style="color:green; font-size:20px;">●</span> <b>Low Risk</b></p>
<p style="margin: 2px 0 12px 24px; font-size: 11px; color: #666;">Low obesity and poverty</p>

<hr style="margin: 10px 0;">
<p style="margin: 0; font-size: 11px; color: #666; text-align: center;">Circle size varies with risk</p>
</div>
'''
m3.get_root().html.add_child(folium.Element(correlation_legend_complete))


m3

In [40]:

 
map_df.to_csv('ma_county_mapping_data.csv', index=False)
print(f"\nExported mapping data: ma_county_mapping_data.csv")


print(f"\nMapping Data Summary:")
print(f"Counties with obesity data: {map_df['obesity_rate'].notna().sum()}")
print(f"Poverty rate range: {map_df['poverty_rate'].min():.1f}% - {map_df['poverty_rate'].max():.1f}%")
if 'obesity_rate' in map_df.columns:
    print(f"Obesity rate range: {map_df['obesity_rate'].min():.1f}% - {map_df['obesity_rate'].max():.1f}%")


Exported mapping data: ma_county_mapping_data.csv

Mapping Data Summary:
Counties with obesity data: 11
Poverty rate range: 0.3% - 6.5%
Obesity rate range: 22.4% - 37.9%
